In [1]:
import pandas as pd
import numpy as np
import ast
import decimal
import psycopg2

In [35]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password=""
)

In [36]:
# Open a cursor to perform database operations
cur = conn.cursor()

In [28]:
users_df = pd.read_csv("C:\\Users\\athar\\OneDrive\\Desktop\\Rutgers\\DBMS\\Final Project\\twitter-analysis2023-dbms694-team10\\users_df.csv")

In [29]:
# Sort dataframe by date column
users_df = users_df.sort_values('date')

In [30]:
users_df.head()

,user_id,name,screen_name,date,twitter_join_date,location,description,verified,followers_count,friends_count,listed_count,favourites_count,preferred_language
0,804046791348015107,Bi Sex Uau,B_King69,2020-04-25 12:21:41+00:00,2016-11-30 19:37:48+00:00,"Acre, Brasil",se for da minha família já pode voltar daq mesmo,False,89,173,0,5446,NaN
1,2242948745,Thomas Krause,tho1965,2020-04-25 12:21:41+00:00,2013-12-25 09:13:33+00:00,NaN,Sportredakteur @nordkurier 🏃‍♂️🚴‍♂️⚽️,False,173,685,9,2184,NaN
2,908326492718764034,शचीन्द्र पाण्डेय,im_S_pandey,2020-04-25 12:21:42+00:00,2017-09-14 13:48:06+00:00,Amethi Uttar Pradesh,Official Twitter Handel Shachindra Pandey (@im...,False,2362,202,3,30668,NaN
3,2929344220,Ralf Schmitz,RusticusArat,2020-04-25 12:21:42+00:00,2014-12-18 10:19:26+00:00,🇩4790 Provinz,"BWLer,ex Offz,Tw meistens zwischen Tür & Angel...",False,778,733,2,32024,NaN
4,1206650133976408064,Büşra Öztaş,schrodingerk42,2020-04-25 12:21:42+00:00,2019-12-16 18:59:53+00:00,NaN,NaN,False,318,220,0,1974,NaN


In [37]:
try:
    for index, row in users_df.iterrows():
        cur.execute("""
        INSERT INTO twitter_users (user_id, name, screen_name, date, twitter_join_date, location, description, verified, followers_count, friends_count, listed_count, favourites_count, preferred_language)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (user_id, date) DO NOTHING
    """, (row['user_id'], row['name'], row['screen_name'], row['date'], row['twitter_join_date'], row['location'], row['description'], row['verified'], row['followers_count'], row['friends_count'], row['listed_count'], row['favourites_count'], row['preferred_language']))

    conn.commit()
    cur.close()
    conn.close()

except Exception as e:
    conn.rollback()
    cur.close()
    conn.close()
    print(e)


In [38]:
# Loop through rows of dataframe
# for i, row in df_u.iterrows():
#     # Check if user already exists in fact table
#     cur.execute("SELECT * FROM User_table_fact WHERE user = %s", (row['user'],))
#     user_fact = cur.fetchone()
    
#     # If user exists, update row in fact table
#     if user_fact:
#         cur.execute("UPDATE User_table_fact SET name = %s, screen_name = %s, date = %s, twitter_join_date = %s, location = %s, description = %s, verified = %s, followers_count = %s, friends_count = %s, listed_count = %s, favourites_count = %s, preferred_language = %s WHERE user_id = %s", (row['name'], row['screen_name'], row['date'], row['twitter_join_date'], row['location'], row['description'], row['verified'], row['followers_count'], row['friends_count'], row['listed_count'], row['favourites_count'], row['preferred_language'], user_fact[0]))
#         user_id = user_fact[0]
#     # If user does not exist, insert row into fact table
#     else:
#         cur.execute("INSERT INTO User_table_fact (user, name, screen_name, date, twitter_join_date, location, description, verified, followers_count, friends_count, listed_count, favourites_count, preferred_language) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) RETURNING user_id", (row['user'], row['name'], row['screen_name'], row['date'], row['twitter_join_date'], row['location'], row['description'], row['verified'], row['followers_count'], row['friends_count'], row['listed_count'], row['favourites_count'], row['preferred_language']))
#         user_id = cur.fetchone()[0]
    
#     # Insert row into dim table
#     cur.execute("INSERT INTO User_table_dim (user_id, date, twitter_join_date, location, description, verified, followers_count, friends_count, listed_count, favourites_count, preferred_language) VALUES (%s, %s, %s, %s, %s, %s, %s, %s